## GCN模型

In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch


# 定义网络结构
class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN,self).__init__()
        self.GCN1 = GCNConv(num_features, 16) # hidden=16, (输入的节点特征，中间隐藏层的维度)
        self.GCN2 = GCNConv(16, num_classes)    # （中间隐藏层的维度，节点类别)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, data):
        # 加载节点特征和邻接关系
        x, edge_index = data.x, data.edge_index
        # 传入卷积层
        x = self.GCN1(x, edge_index)
        x = F.relu(x)  # ReLU激活函数
        x = self.dropout(x)  #dropout层,防止过拟合
        x = self.GCN2(x, edge_index)  # 第二个卷积层
        #将经过两层卷积层得到的特征输入log_softmax函数得到概率分布
        return F.log_softmax(x, dim=1)


## GAT模型

In [ ]:
from torch_geometric.nn import GATConv
import torch.nn.functional as F
import torch

class GAT(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GAT,self).__init__()
        self.GAT1 = GATConv(num_features, 8, heads = 8, concat = True, dropout = 0.6)
        self.GAT2 = GATConv(8*8, num_classes, dropout = 0.6)  
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.GAT1(x, edge_index)
        x = F.relu(x)
        x = self.GAT2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

## GraphSAGE


In [ ]:
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
import torch


class GraphSAGE(torch.nn.Module):

    def __init__(self, num_features, hidden_dim, num_classes):
        super(GraphSAGE, self).__init__()
        self.sage1 = SAGEConv(num_features, hidden_dim)  # 定义两层GraphSAGE层
        self.sage2 = SAGEConv(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.sage1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.sage2(x, edge_index)

        return F.log_softmax(x, dim=1)

## GIN模型

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINConv, global_add_pool
from torch.nn import Sequential, Linear, ReLU

class GIN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GIN, self).__init__()
        self.conv1 = GINConv(Sequential(Linear(in_channels, 64), ReLU(), Linear(64, 64)))
        self.conv2 = GINConv(Sequential(Linear(64, 64), ReLU(), Linear(64, out_channels)))
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = F.relu(self.conv1(x, edge_index))
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

## train

In [ ]:
def train(model, data,device):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
    loss_function = torch.nn.CrossEntropyLoss().to(device)
    model.train()
    num_epochs = epochs
    for epoch in range(num_epochs): # 200 epochs
        out_d= model(data)
        optimizer.zero_grad()
        loss = loss_function(out_d[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        print('Epoch {:03d} loss {:.4f}'.format(epoch, loss.item()))

## test

In [ ]:
def test(model, data):
    model.eval()
    _, pred = model(data).max(dim=1)
    correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    acc = correct / int(data.test_mask.sum())
    print( 'Accuracy: {:.4f}'.format(acc))

## show_time

In [ ]:
import time
def show_time():
    t0 = time.time()
    r = 0
    for i in range(10000000):
        r += i
    time.sleep(2)
    # print(r)
    t1 = time.time()
    spend1 = t1 - t0

    # print("-------------------------------")
    print("运行时间：{}s".format(spend1))
    print("测试完毕")


import pickle
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.loader import NeighborSampler

def load_local_planetoid_dataset(name, root='../dataset', transform=None):
    """
    加载本地Planetoid数据集（CiteSeer, Cora, Flickr）
    """
    # 加载本地pkl文件
    data_dict = pickle.load(open(f'{root}/{name}/{name.lower()}.pkl', 'rb'))
    
    print(f"Loading {name} dataset from local file:")
    print(f"  Nodes: {data_dict['topo'].shape[0]}")
    print(f"  Edges: {data_dict['topo'].nnz}")
    print(f"  Features: {data_dict['attr'].shape[1]}")
    print(f"  Classes: {len(np.unique(data_dict['label']))}")
    
    # 转换为torch tensors
    edge_index = torch.tensor(np.array(data_dict['topo'].nonzero()), dtype=torch.long)
    x = torch.tensor(data_dict['attr'].toarray(), dtype=torch.float)
    y = torch.tensor(data_dict['label'], dtype=torch.long)
    
    # 创建PyTorch Geometric Data对象
    data = Data(x=x, edge_index=edge_index, y=y)
    
    # 应用变换
    if transform:
        data = transform(data)
    
    # 创建一个简单的Dataset类来模拟Planetoid接口
    class LocalPlanetoidDataset:
        def __init__(self, data):
            self.data = data
            self.num_features = data.x.size(1)
            self.num_classes = data.y.unique().size(0)
            self.num_node_features = data.x.size(1)
            
        def __len__(self):
            return 1
            
        def __getitem__(self, idx):
            return self.data
    
    return LocalPlanetoidDataset(data)

# 加载Cora数据集（默认）
dataset = load_local_planetoid_dataset('Cora', transform=NormalizeFeatures())

# 加载CiteSeer数据集
# dataset = load_local_planetoid_dataset('CiteSeer', transform=NormalizeFeatures())

# 加载Flickr数据集
# dataset = load_local_planetoid_dataset('Flickr', transform=NormalizeFeatures())

data = dataset[0]

In [ ]:
from torch_geometric.datasets import Planetoid, Flickr
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.loader import NeighborSampler

# 加载Cora数据集
dataset = Planetoid(root='./dataset', name="Cora", transform=NormalizeFeatures())


# 加载CiteSeer数据集
# dataset = Planetoid(root='./dataset', name="CiteSeer", transform=NormalizeFeatures())


# 加载Flickr数据集
# dataset = Flickr(root='./dataset/Flickr')   

data = dataset[0]

In [ ]:
epochs = 10
lr = 1e-3
weight_decay = 5e-3
momentum = 0.5
hidden_dim = 128
output_dim = 7
num_features = dataset.num_features
num_classes = dataset.num_classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = GCN(num_features,num_classes).to(device)
# model = GAT(num_features, num_classes).to(device)
model =GraphSAGE(num_features, hidden_dim,num_classes).to(device)
#model = GIN(num_features, num_classes).to(device)
train(model, data, device)
test(model, data)
show_time()

In [ ]:
# Sampler采样
print("Sampler采样:.................")
train_loader = NeighborSampler(data.edge_index, sizes=[25, 10], batch_size=64, shuffle=True)
model = GCN(num_features, num_classes).to(device)
def train_with_sampling(model, data, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch_size, n_id, adjs in train_loader:
        # adjs 是 [(edge_index, e_id, size), ...] 的形式，其中每个 edge_index 是 [2, num_messages] 的格式
        adjs = [adj.to(device) for adj in adjs]
        optimizer.zero_grad()

        # 通过 n_id[:batch_size] 选择原始的中心节点
        out = model(data.x[n_id].to(device), adjs[0].edge_index)
        
        # 只使用中心节点对应的输出进行损失计算
        loss = F.nll_loss(out[:batch_size], data.y[n_id[:batch_size]].to(device))
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch_size
    return total_loss / len(train_loader.dataset)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# 子图采样训练
for epoch in range(1, 201):
    loss = train_with_sampling(model, data, train_loader, optimizer, device)
    if epoch % 10 == 0:
        accuracy = test(model, data, device)
        print(f'(Subgraph Sampling) Epoch: {epoch:03d}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')